In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn
from tensorflow import keras
from matplotlib import pyplot
import imblearn
import gensim
import nltk
from pathlib import Path
import time
import tensorflow_hub as hub
import tqdm

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
#!jupyter nbextension enable --py widgetsnbextensio

In [3]:
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [4]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [5]:
df_date = pd.read_csv('/storage/data/ad_matched_queries_20190831__by_date.csv')
df_date.head(5)

,campaign_label,date,source_medium,campaign,ad_group,ad_matched_query,ad_cost,impressions,clicks,users,new_users,sessions,bounces,created_accounts,transactions,transaction_revenue
0,brand,2017-03-07,google / cpc,Brand Terms,Paperspace For Teams,paper space,0.31,4,1,1,1,1,0,0,0,0.0
1,brand,2017-03-07,google / cpc,Brand Terms,Paperspace For Teams,paperspace,1.06,28,5,4,3,4,2,0,0,0.0
2,brand,2017-03-08,google / cpc,Brand Terms,Paperspace For Teams,paperspace,0.94,24,3,1,0,1,0,0,0,0.0
3,brand,2017-03-09,google / cpc,Brand Terms,Paperspace For Teams,paperspace,1.70,50,4,2,2,2,1,0,0,0.0
4,brand,2017-03-10,google / cpc,Brand Terms,Paperspace For Teams,paperspace,4.57,49,7,5,4,9,0,1,1,6.0


In [6]:
REPORT_SOURCE = 'google/ads'
REPORT_ID = 'InsightFall2019 Ad Group Keyword 20191001'
FILENAME = 'aggregate_20190831.csv.gz'

PATH_DATA = Path('/') / 'storage' / 'data' / REPORT_SOURCE / REPORT_ID / FILENAME
df_cat = pd.read_csv(PATH_DATA)
df_cat.head(5)


,Campaign ID,Ad group ID,Keyword ID,Campaign,Ad group,Search keyword,Keyword/Placement,Search keyword match type,Campaign type,Campaign bid strategy type,...,Currency,Cost,Impressions,Clicks,Conversions,Conv. value,Avg. position,Impr. (Abs. Top) %,Impr. (Top) %,Search Impr. share
0,765732120,39460828726,44160165835,General Product Terms,Cloud Virtual Desktop - Enterprise Market,+Cloud +Virtual +Desktop,+Cloud +Virtual +Desktop,Broad,Search Only,Target CPA,...,USD,0.0,4,0,0.0,0.0,1.5,0.0,0.0,0.3636
1,765732120,39616922879,341042909967,General Product Terms,VDI Cloud,+vdi in the +cloud,+vdi in the +cloud,Broad,Search Only,Target CPA,...,USD,0.0,1,0,0.0,0.0,7.0,0.0,0.0,1.0000
2,765732120,43344958467,27994524387,General Product Terms,Cloud Virtual Machine - Enterprise Market,+Cloud +Virtual +Machine,+Cloud +Virtual +Machine,Broad,Search Only,Target CPA,...,USD,0.0,2,0,0.0,0.0,2.0,0.0,0.0,1.0000
3,766802375,39100138983,28912106981,Competitor Terms,VMware Cloud Computing,+cloud +vmware,+cloud +vmware,Broad,Search Only,Enhanced CPC,...,USD,0.0,1,0,0.0,0.0,3.0,0.0,0.0,0.5000
4,765732120,39616922879,32957936454,General Product Terms,VDI Cloud,+vdi +cloud,+vdi +cloud,Broad,Search Only,Target CPA,...,USD,0.0,3,0,0.0,0.0,1.0,0.0,0.0,1.0000


In [7]:
df_cat.rename(columns={'Ad group':'ad_group'}, inplace=True)
df_cat1 = df_cat[['ad_group','Campaign', 'Search keyword', 'Search keyword match type', 'Campaign type', 'Campaign bid strategy type']].copy()
df_cat1.head(5)

,ad_group,Campaign,Search keyword,Search keyword match type,Campaign type,Campaign bid strategy type
0,Cloud Virtual Desktop - Enterprise Market,General Product Terms,+Cloud +Virtual +Desktop,Broad,Search Only,Target CPA
1,VDI Cloud,General Product Terms,+vdi in the +cloud,Broad,Search Only,Target CPA
2,Cloud Virtual Machine - Enterprise Market,General Product Terms,+Cloud +Virtual +Machine,Broad,Search Only,Target CPA
3,VMware Cloud Computing,Competitor Terms,+cloud +vmware,Broad,Search Only,Enhanced CPC
4,VDI Cloud,General Product Terms,+vdi +cloud,Broad,Search Only,Target CPA


In [8]:
df_searchwords = df_cat1[df_cat1['ad_group'] == 'VDI Cloud']
#print(df_searchwords.head(100))
print(df_searchwords['Search keyword'].unique())

['+vdi in the +cloud' '+vdi +cloud' '+cloud +vdi' 'vdi in the cloud'
 '+vdi +in +cloud' '+vdi +on +cloud']


In [9]:
df_unique = df_cat.groupby(['ad_group','Search keyword', 'Search keyword match type']).size().reset_index().rename(columns={0:'count'})
df_unique.head(25)

,ad_group,Search keyword,Search keyword match type,count
0,3ds Max,+3ds +max +mac +os,Broad,4
1,3ds Max,+3ds +max +mac +version,Broad,1
2,3ds Max,+autodesk +3ds +max +mac,Broad,14
3,3ds Max,+mac +3ds +max,Broad,66
4,3ds Max,3ds max for mac,Exact,57
5,3ds Max,3ds max for mac os,Exact,1
6,3ds Max,3ds max mac,Exact,50
7,3ds Max,3ds max mac os x,Exact,2
8,3ds Max,autodesk 3ds max for mac,Exact,6
9,AI Cloud Amazon_broad,+amazon +cloud +ai,Broad,78


In [10]:
srch_keywords_dict = {}
for name, group in df_unique.groupby('ad_group'):
    values = (' '.join(map(str, group['Search keyword'].values)))
    unique_values = ' '.join(list(set(values.split(' '))))
    srch_keywords_dict[name] = unique_values

srch_keywords_df = pd.DataFrame.from_dict(srch_keywords_dict, orient='index').reset_index()
srch_keywords_df.rename(columns={'index': "ad_group", 0: "srch_keywords"}, inplace=True)

print(srch_keywords_df.shape)
filtered_df = srch_keywords_df[srch_keywords_df['ad_group'].str.endswith('_broad') | srch_keywords_df['ad_group'].str.endswith('_exact') | srch_keywords_df['ad_group'].str.endswith('_phrase')].copy()
print(filtered_df)


#use campaign as another variable to split the stuff on
#we are filtering out 1/3 of the data because we can't join on type of match

#IDEAL
#match specific set of search keywords within ad group that were used for the ad_matched_query
#pulling in the ad group id or keyword id into ad_match_queries dataset

#TODO
#create embedding of ad_match_query
#create embedding of search keyword, one hot encode match type

(386, 2)
                          ad_group  \
1            AI Cloud Amazon_broad   
2            AI Cloud Google_broad   
3               AI Cloud Gpu_broad   
4         AI Cloud Microsoft_broad   
5                   AI Cloud_broad   
..                             ...   
381          Windows On Mac_phrase   
382  Windows Programs On Mac_exact   
383       Windows VM On Mac_phrase   
384  iPython Notebook Online_exact   
385  iPython Notebook Server_exact   

                                         srch_keywords  
1    +aws +intelligence +ai +artificial +cloud +amazon  
2         +intelligence +google +ai +artificial +cloud  
3                                      +gpu +cloud +ai  
4      +intelligence +ai +artificial +cloud +microsoft  
5                 +intelligence +cloud +ai +artificial  
..                                                 ...  
381                                 for mac windows on  
382                        mac run on programs windows  
383                  

In [11]:
df_join = pd.merge(filtered_df, df_date, on='ad_group')

df_join.head()

,ad_group,srch_keywords,campaign_label,date,source_medium,campaign,ad_matched_query,ad_cost,impressions,clicks,users,new_users,sessions,bounces,created_accounts,transactions,transaction_revenue
0,AI Cloud Amazon_broad,+aws +intelligence +ai +artificial +cloud +amazon,ml,2018-11-29,google / cpc,2018_NA-US_en_ML_search_desktop,amazon cloud ai,2.49,1,1,1,0,1,1,0,0,0.0
1,AI Cloud Google_broad,+intelligence +google +ai +artificial +cloud,ml,2018-10-01,google / cpc,2018_NA-US_en_ML_search_desktop,google cloud ai client,19.79,2,1,1,1,1,1,0,0,0.0
2,AI Cloud Google_broad,+intelligence +google +ai +artificial +cloud,ml,2018-10-02,google / cpc,2018_NA-US_en_ML_search_desktop,google cloud ai,15.00,18,1,1,1,1,1,0,0,0.0
3,AI Cloud Google_broad,+intelligence +google +ai +artificial +cloud,ml,2018-10-04,google / cpc,2018_NA-US_en_ML_search_desktop,google cloud ai,0.00,1,0,0,0,0,0,0,0,0.0
4,AI Cloud Google_broad,+intelligence +google +ai +artificial +cloud,ml,2018-10-22,google / cpc,2018_EUR-W-CH_en_ML_search_desktop,google cloud artificial intelligence,2.43,1,1,0,0,0,0,0,0,0.0


In [12]:
list_labels = df_join['clicks'].tolist()
clicks = df_join['clicks']
click_counts = df_join['clicks'].value_counts()
print(len(list_labels))

df_join['click_thru_rate'] = df_join['clicks'] / df_join['impressions']
click_thru_rate = df_join['click_thru_rate']
click_thru_rate.to_pickle('click_thru_rate.pkl')

clicks.to_pickle("clicks.pkl")

165051


In [13]:
#create match type column
df_join['match_type'] = np.nan
df_join['match_type'] = pd.np.where(df_join['ad_group'].str.contains("broad"), "broad",
                        pd.np.where(df_join['ad_group'].str.contains("exact"), "exact",
                        pd.np.where(df_join['ad_group'].str.contains("phrase"), "phrase", 'ERROR')))
df_join.match_type.unique()

encoded_columns = pd.get_dummies(df_join['match_type'])
df_join = df_join.join(encoded_columns).drop('match_type', axis=1)
df_join.head(5)

,ad_group,srch_keywords,campaign_label,date,source_medium,campaign,ad_matched_query,ad_cost,impressions,clicks,...,new_users,sessions,bounces,created_accounts,transactions,transaction_revenue,click_thru_rate,broad,exact,phrase
0,AI Cloud Amazon_broad,+aws +intelligence +ai +artificial +cloud +amazon,ml,2018-11-29,google / cpc,2018_NA-US_en_ML_search_desktop,amazon cloud ai,2.49,1,1,...,0,1,1,0,0,0.0,1.000000,1,0,0
1,AI Cloud Google_broad,+intelligence +google +ai +artificial +cloud,ml,2018-10-01,google / cpc,2018_NA-US_en_ML_search_desktop,google cloud ai client,19.79,2,1,...,1,1,1,0,0,0.0,0.500000,1,0,0
2,AI Cloud Google_broad,+intelligence +google +ai +artificial +cloud,ml,2018-10-02,google / cpc,2018_NA-US_en_ML_search_desktop,google cloud ai,15.00,18,1,...,1,1,1,0,0,0.0,0.055556,1,0,0
3,AI Cloud Google_broad,+intelligence +google +ai +artificial +cloud,ml,2018-10-04,google / cpc,2018_NA-US_en_ML_search_desktop,google cloud ai,0.00,1,0,...,0,0,0,0,0,0.0,0.000000,1,0,0
4,AI Cloud Google_broad,+intelligence +google +ai +artificial +cloud,ml,2018-10-22,google / cpc,2018_EUR-W-CH_en_ML_search_desktop,google cloud artificial intelligence,2.43,1,1,...,0,0,0,0,0,0.0,1.000000,1,0,0


In [14]:
#encode date
from datetime import date

#Transform string to date
df_join['date'] = pd.to_datetime(df_join['date'])

#Extracting Year
df_join['year'] = df_join['date'].dt.year

#Extracting Month
df_join['month'] = df_join['date'].dt.month

#Extracting Day
df_join['day'] = df_join['date'].dt.day

#Extracting the weekday name of the date
df_join['day_name'] = df_join['date'].dt.day_name()

#Extracting passed years since the date
df_join['passed_years'] = date.today().year - df_join['date'].dt.year

#Extracting passed months since the date
df_join['passed_months'] = (date.today().year - df_join['date'].dt.year) * 12 + date.today().month - df_join['date'].dt.month

#df_join[['date','year','month','day','day_name','passed_years','passed_months']]

In [15]:
# clean keywords
from nltk.stem import PorterStemmer
df_join["srch_keywords"].str.lower().str.replace('[!"#$%&()*+-/:;<=>?@[\\]^_`{|}~]',' ')
df_join["srch_keywords"].str.replace("[0-9]", ' ')
df_join["srch_keywords"].dropna()
df_join["srch_keywords"] = df_join["srch_keywords"].apply(lambda x:' '.join(x.split()))
#unique_words = df_append.unique()

df_join["srch_keywords"] = df_join["srch_keywords"].str.replace('+','')

ps = PorterStemmer()
df_join["srch_keywords"].map(ps.stem)

print(df_join["srch_keywords"].shape)
df_join["srch_keywords"].head()
srch_keywords = df_join["srch_keywords"]

(165051,)


In [16]:
#load ELMo from tensorflow hub
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [17]:
# test
tokens_input = [["the", "cat", "is", "on", "the", "mat"],
                ["dogs", "are", "in", "the", "fog", ""]]
tokens_length = [6, 5]
embeddings = elmo(
    inputs={
        "tokens": tokens_input,
        "sequence_len": tokens_length
    },
    signature="tokens",
    as_dict=True)["elmo"]
embeddings.shape

TensorShape([Dimension(2), Dimension(6), Dimension(1024)])

In [ ]:
# Create a custom layer that allows us to update weights (lambda layers do not have trainable parameters!)

import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer

class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))

        self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [ ]:
#input_text = layers.Input(shape=(1,), dtype="string")
#embedding = ElmoEmbeddingLayer()(input_text)

In [18]:
def elmo_vectors(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

  print(embeddings)
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [ ]:
list_keywords = [srch_keywords[i:i+100] for i in range(0,srch_keywords.shape[0],100)]
elmo_keywords = [elmo_vectors(x) for x in list_keywords]

In [ ]:
elmo_keywords_array = np.concatenate(elmo_keywords, axis = 0)
elmo_keywords_array.shape

In [ ]:
import pickle

pickle_out = open("elmo_keywords.pickle","wb")
pickle.dump(elmo_keywords_array, pickle_out)
pickle_out.close()

In [ ]:
#keyword simple text embeddings
from tensorflow.keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence

t = Tokenizer(num_words=20000)
t.fit_on_texts(df_join["srch_keywords"])

#print(t.word_counts)
print(t.document_count)
#print(t.word_index)

sequenced_keywords = t.texts_to_sequences(df_join["srch_keywords"])
max_length = len(max(sequenced_keywords, key=len))
padded_keywords = sequence.pad_sequences(sequenced_keywords, maxlen=max_length)
print(padded_keywords.shape)

#df_join['keyword_padded'] = padded_sequences
#encoded_queries = t.texts_to_matrix(df_BYDATE['ad_matched_query'], mode='binary')
#print(encoded_queries)

In [ ]:
#corpus: combine keywords and queries
#corpus = df_join["srch_keywords"].map(str) + ' ' + df_join["ad_matched_query"].map(str)
#corpus = corpus.apply(lambda x: ' '.join(pd.unique(x.split())))

In [19]:
# clean queries
from nltk.stem import PorterStemmer
df_join["ad_matched_query"].str.lower().str.replace('[!"#$%&()*+-/:;<=>?@[\\]^_`{|}~]',' ')
df_join["ad_matched_query"].str.replace("[0-9]", ' ')
df_join["ad_matched_query"].dropna()
df_join["ad_matched_query"] = df_join["ad_matched_query"].apply(lambda x:' '.join(x.split()))
#unique_words = df_append.unique()

df_join["ad_matched_query"] = df_join["ad_matched_query"].str.replace('+','')

ps = PorterStemmer()
df_join["ad_matched_query"].map(ps.stem)
df_join["ad_matched_query"].head()
ad_matched_query = df_join["ad_matched_query"]
# tokenize by letter
#df_date['stemmed'] = df_date['ad_matched_query'].apply(lambda x: [stemmer.stem(x) for y in x])

In [20]:
import pickle

list_queries1 = [ad_matched_query[i:i+1000] for i in range(0,80000,1000)]
elmo_queries1 = [elmo_vectors(x) for x in list_queries1]
elmo_queries_array1 = np.concatenate(elmo_queries1, axis = 0)

pickle_out = open("elmo_queries_array1.pickle","wb")
pickle.dump(elmo_queries_array1, pickle_out)
pickle_out.close()

Tensor("module_apply_default/aggregation/mul_3:0", shape=(1000, 24, 1024), dtype=float32)
Tensor("module_apply_default_1/aggregation/mul_3:0", shape=(1000, 20, 1024), dtype=float32)
Tensor("module_apply_default_2/aggregation/mul_3:0", shape=(1000, 17, 1024), dtype=float32)
Tensor("module_apply_default_3/aggregation/mul_3:0", shape=(1000, 11, 1024), dtype=float32)
Tensor("module_apply_default_4/aggregation/mul_3:0", shape=(1000, 10, 1024), dtype=float32)
Tensor("module_apply_default_5/aggregation/mul_3:0", shape=(1000, 7, 1024), dtype=float32)
Tensor("module_apply_default_6/aggregation/mul_3:0", shape=(1000, 7, 1024), dtype=float32)
Tensor("module_apply_default_7/aggregation/mul_3:0", shape=(1000, 4, 1024), dtype=float32)
Tensor("module_apply_default_8/aggregation/mul_3:0", shape=(1000, 6, 1024), dtype=float32)
Tensor("module_apply_default_9/aggregation/mul_3:0", shape=(1000, 12, 1024), dtype=float32)
Tensor("module_apply_default_10/aggregation/mul_3:0", shape=(1000, 13, 1024), dtype=fl

In [21]:
import pickle

list_queries2 = [ad_matched_query[i:i+1000] for i in range(80000,ad_matched_query.shape[0],1000)]
elmo_queries2 = [elmo_vectors(x) for x in list_queries2]
elmo_queries_array2 = np.concatenate(elmo_queries2, axis = 0)

pickle_out = open("elmo_queries_array2.pickle","wb")
pickle.dump(elmo_queries_array2, pickle_out)
pickle_out.close()

Tensor("module_apply_default_80/aggregation/mul_3:0", shape=(1000, 23, 1024), dtype=float32)
Tensor("module_apply_default_81/aggregation/mul_3:0", shape=(1000, 13, 1024), dtype=float32)
Tensor("module_apply_default_82/aggregation/mul_3:0", shape=(1000, 25, 1024), dtype=float32)
Tensor("module_apply_default_83/aggregation/mul_3:0", shape=(1000, 19, 1024), dtype=float32)
Tensor("module_apply_default_84/aggregation/mul_3:0", shape=(1000, 8, 1024), dtype=float32)
Tensor("module_apply_default_85/aggregation/mul_3:0", shape=(1000, 8, 1024), dtype=float32)
Tensor("module_apply_default_86/aggregation/mul_3:0", shape=(1000, 11, 1024), dtype=float32)
Tensor("module_apply_default_87/aggregation/mul_3:0", shape=(1000, 8, 1024), dtype=float32)
Tensor("module_apply_default_88/aggregation/mul_3:0", shape=(1000, 4, 1024), dtype=float32)
Tensor("module_apply_default_89/aggregation/mul_3:0", shape=(1000, 4, 1024), dtype=float32)
Tensor("module_apply_default_90/aggregation/mul_3:0", shape=(1000, 18, 1024

In [23]:
elmo_queries_array1.shape

NameError: name 'elmo_queries_array1' is not defined

In [ ]:
query_embeddings = get_word2vec_embeddings(word2vec, df_join["ad_matched_query"])
print(len(query_embeddings))

In [ ]:
#query simple text embeddings
from tensorflow.keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence

#t = Tokenizer(num_words=20000)
t.fit_on_texts(df_join["ad_matched_query"])
word_index = t.word_index
#print(t.word_counts)
#print(t.document_count)
#print(t.word_index)

sequenced_queries = t.texts_to_sequences(df_join["ad_matched_query"])
max_length = len(max(sequenced_queries, key=len))
padded_queries = sequence.pad_sequences(sequenced_queries, maxlen=max_length)
print(padded_queries.shape)

#df_join['keyword_padded'] = padded_sequences
#encoded_queries = t.texts_to_matrix(df_BYDATE['ad_matched_query'], mode='binary')
#print(encoded_queries)

In [ ]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()# Fit your data on the scaler object

df_join[['ad_cost']] = scaler.fit_transform(df_join[['ad_cost']])

In [ ]:
df_features = df_join.drop(columns=['campaign_label', 'source_medium', 'impressions', 'transaction_revenue', 'campaign', \
                                    'users', 'new_users', 'sessions', 'bounces', 'transactions', 'created_accounts'], axis=1)
df_features.head(5)

In [ ]:
keyword_array = np.asarray(keyword_embeddings)
print(keyword_array.shape)
query_array = np.asarray(query_embeddings)
print(query_array.shape)
text_array = np.concatenate((keyword_array,query_array),axis=1)
print(text_array.shape)
print(type(text_array))

In [ ]:
df_features2 = pd.concat([df_features, pd.DataFrame(text_array)], axis=1)


In [ ]:
df_final = df_features2.drop(columns=['ad_group', 'srch_keywords', 'ad_matched_query', 'clicks'], axis=1)
df_final.head(5)

In [ ]:
pd.to_pickle(df_final, 'df_final.pickle')


In [ ]:

#create model
model = Sequential()#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(132040, 613,1)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model_cnn = model.fit(x_train_w2v, y_train_w2v, validation_data=(x_test_w2v, y_test_w2v), epochs=3)

In [ ]:
#TFIDF vectorize queries
#from sklearn.feature_extraction.text import TfidfVectorizer
#corpus = [line.split(',') for line in df_BYDATE['ad_matched_query']]
#n_features = 2500
#vectorizer = TfidfVectorizer(ngram_range=(1, 2),stop_words='english', lowercase=True ,max_df=0.8, min_df=10, max_features=n_features)
#x = vectorizer.fit_transform(df_join['ad_matched_query'])
#vocab = vectorizer.vocabulary_
#print(vocab)
#print(vectorizer.idf_)
##df_join['queries_vect']=list(x.toarray())
#print(x.shape)

#corpus_array


In [ ]:
#pd.to_pickle(x, 'TFIDF_vec.pickle')

In [ ]:
#pd.to_pickle(df_join, 'df_non_text_features.pickle')

In [ ]:
print(fit_features.shape)
pd.to_pickle(fit_features, 'fit_features.pickle')